<a href="https://colab.research.google.com/github/sitinoldae/ElectronProcessor/blob/main/pdf2csvAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os #for navigating folder structures and running command line tools ghostscript and tesseract
import matplotlib.pyplot as plt #for visualizing steps in this notebook
from PIL import Image #for visualizing results
import tempfile
import numpy as np #for some image manipulation

In [ ]:
tempDir=tempfile.mkdtemp(dir=os.getcwd())

In [ ]:
import os

directory_path = '/content/examplePDFs'

# Check if the directory exists, if not create it
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# List the contents of the directory
directory_contents = os.listdir(directory_path)
print(directory_contents)
!apt-get install -y ghostscript
temp_dir = "/content/temp_images"
os.makedirs(temp_dir, exist_ok=True)
import subprocess
pdf_file_path="/content/examplePDFs/input2.pdf"
output_template = os.path.join(temp_dir, "page-%03d.png")
command = ["gs", "-sDEVICE=pngalpha", f"-sOutputFile={output_template}", "-dBATCH", "-dNOPAUSE", "-r150", pdf_file_path]

result = subprocess.run(command, capture_output=True, text=True)

if result.stderr:
    print("Errors:", result.stderr)
else:
    print("Image Extraction completed !")


['input2.pdf']
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.4).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
PDF conversion completed successfully!


In [ ]:
!pip install tqdm
!apt install tesseract-ocr
!pip install pytesseract

In [ ]:
import pytesseract
from PIL import Image
from tqdm.notebook import tqdm

def extract_text_from_image(image_path, margin=70): # default margin set to 20 pixels
    image = Image.open(image_path)

    # Get the dimensions of the image
    width, height = image.size

    # Define a box to crop from (left, upper, right, lower)
    left = margin
    upper = margin
    right = width - margin
    lower = height - margin

    # Crop the image
    cropped_image = image.crop((left, upper, right, lower))

    # Extract text from the cropped image
    extracted_text = pytesseract.image_to_string(cropped_image)
    return extracted_text

# List all images in the temp directory
image_files = sorted([f for f in os.listdir(temp_dir) if f.endswith('.png')])
image_paths = [os.path.join(temp_dir, image_file) for image_file in image_files]

# Ask user for the number of pages to skip at the start and end
num_skip_start = int(input("Enter the number of pages to skip at the start: "))
num_skip_end = int(input("Enter the number of pages to skip at the end: "))

# Adjust the image paths based on user input
image_paths = image_paths[num_skip_start:len(image_paths)-num_skip_end]

# Create a progress bar
pbar = tqdm(total=len(image_paths), desc="Processing Images", unit="image")

# Process each image and update progress bar
extracted_texts = []
for image_path in image_paths:
    extracted_text = extract_text_from_image(image_path)
    extracted_texts.append(extracted_text)
    pbar.set_postfix(file=image_path.split('/')[-1], refresh=True) # display the filename being processed
    pbar.update(1)

# Close the progress bar
pbar.close()

# Combine all the extracted texts
extractedTextFull = "\n\n".join(extracted_texts)

print("Text extraction completed!")


Enter the number of pages to skip at the start: 2
Enter the number of pages to skip at the end: 50


Processing Images:   0%|          | 0/2 [00:00<?, ?image/s]

In [ ]:
extractedTextFull='\n'.join([elm.strip() for elm in extractedTextFull.split('\n') if len(elm.strip())>0])
print(extractedTextFull)

Section No and Name : 1-Tondiarpet Ward No 38 N S K Street Nedunchezhiyan Nagar
7
2
WaD0300335 wap0688523 3 WaD0616243
Name: Mathavi - Name: Suseela - Name: Anandhan -
Husband's Name : Murali - Husband's Name: Gnanam - Father's Name : Arumugam -
House Number : 1 House Number : 2 House Number : 2
Age: 31 Gender: FEMALE Photo is || Age: 69 Gender: FEMALE Photois || Age: 49 Gender: MALE Photo is
Available Available Available
4 wap0688499 a wap0688275 6 wab1007541
Name: Venkatesan - Name: Kala - Name: Saranya-
Father's Name: sammantham - Husband's Name : Venkatesan - Father's Name : Anandhan -
House Number : 2 House Number : 2 House Number : 2
Age: 48 Gender: MALE Photo is Age: 45 Gender: FEMALE Photo is Age: 26 Gender: FEMALE Photo is
Available Available Available
Z wab1487701 8 wab1670462 8 wab1670819
Name: Mahalakshmi - Name : Sowmiya - Name: Sankarasubbu -
Father's Name : Sekar - Father's Name : Sekar - Father's Name : Vivekanandhan -
House Number : 2 House Number : 2 House Number : 3A

In [ ]:
import re

def filter_short_lines(text):
    # Remove numbers with the exact format of four digits
    text = re.sub(r'\b\d{4}\b', '', text)

    # Split the text into lines
    lines = text.split('\n')

    # Replace lines shorter than 35 characters or containing the specified words with a blank line
    unwanted_words = ["Available", "Photo", "is"]
    new_lines = [line if len(line) >= 35 and all(word not in line for word in unwanted_words) else '' for line in lines]

    # Join the lines back into a single string
    return '\n'.join(new_lines)

filteredText = filter_short_lines(extractedTextFull)
print(filteredText)




Section No and Name : 1-Tondiarpet Ward No 38 N S K Street Nedunchezhiyan Nagar





































































































































































































































































































































1 WaD0300335 2 wap0688523 3 WaD0616243
Name: Mathavi - Name: Suseela - Name: Anandhan -
Husband's Name : Murali - Husband's Name: Gnanam - Father's Name : Arumugam -
House Number : 1 House Number : 2 House Number : 2


4 wap0688499 a wap0688275 6 wab1007541
Name: Venkatesan - Name: Kala - Name: Saranya-
Father's Name: sammantham - Husband's Name : Venkatesan - Father's Name : Anandhan -
House Number : 2 House Number : 2 House Number : 2


Z wab1487701 8 wab1670462 8 wab1670819
Name: Mahalakshmi - Name : Sowmiya - Name: Sankarasubbu -
Father's Name : Sekar - Father's Name : Sekar - Fathe

In [ ]:
print(filteredText[:250])  # print the first 1000 characters
import pandas as pd
import re


# Updated regex patterns for extracting voter details
voter_pattern = (
    r"(?P<voter_id>[A-Za-z0-9]+)\s?"  # Voter ID
    r"Name: (?P<name>[A-Za-z\s\-]+) -"  # Name
    r"(?:Husband's Name|Father's Name) : (?P<relation_name>[A-Za-z\s\-]+) -"  # Relation's Name
    r"House Number : (?P<house_number>\d+)"  # House Number
    r"Age: (?P<age>\d+) Gender: (?P<gender>MALE|FEMALE)"  # Age and Gender
)

matches = re.finditer(voter_pattern, filteredText)

# Populate a DataFrame
df = pd.DataFrame({
    "Voter ID": [match.group("voter_id").strip() for match in matches],
    "Name": [match.group("name").strip() for match in matches],
    "Relation's Name": [match.group("relation_name").strip() for match in matches],
    "House Number": [match.group("house_number").strip() for match in matches],
    "Age": [int(match.group("age").strip()) for match in matches],
    "Gender": [match.group("gender").strip() for match in matches],
})

print(df)

Assembly Constituency No and Name : 11-Dr.Radhakrishnan Nagar
Section No and Name : 4-Tondiarpet Ward No 38 Chinnaswamy Street Nedunchezian Nagar
Part number : 1


WaQD1007335 wab0452813 604 WaD0452557
Name: Shalini - Name: Syed Ameer John - Name: Fa
Empty DataFrame
Columns: [Voter ID, Name, Relation's Name, House Number, Age, Gender]
Index: []
